# 한빛아카데미 - 새로나온책 10 페이지
- https://www.hanbit.co.kr/academy/books/new_book_list.html
- 제목, 저자, 출간일, 쪽수, 가격, 역자

In [27]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm #진척도를 알려주는 것 

In [28]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
url = base_url + sub_url + '?page=' + str(1)
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

### 1. 찾으려고 하는 목록 데이터 

In [29]:
lis = soup.select('li.sub_book_list')
len(lis)

20

### 2. 하나의 데이터로 원하는 정보 추출

In [30]:
li = lis[0]
href = li.find('a')['href']
href

'/academy/books/book_view.html?p_code=B8672666879'

- sub page로 이동하여 원하는 정보 추출

In [31]:
sub_req = requests.get(base_url + href)
sub_soup = BeautifulSoup(sub_req.text,'html.parser')

In [32]:
title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
title

'STEM CookBook, 문제 해결력을 키우는 기계설계'

In [33]:
info_lis = sub_soup.select('.info_list > li')
info_dict = {}
for li in info_lis:
  key = li.find('strong').get_text().split(' ')[0]
  val = li.find('span').get_text().strip()
  info_dict[key] =val

info_dict

{'ISBN': '9791156646150',
 '물류코드': '4615',
 '저자': '이상범',
 '출간': '2022-08-05',
 '페이지': '680 쪽'}

In [34]:
author = info_dict['저자']
try:
    translator = info_dict['번역']
except:
    translator = '-'
date = info_dict['출간']
page = int(info_dict['페이지'].split(' ')[0].replace(',', ''))
author, translator, date, page

('이상범', '-', '2022-08-05', 680)

In [35]:
# 가격정보
price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',', ''))
price

34000

### 3. 한 페이지내의 모든 정보 가져오기

In [41]:
lines = []
for li in lis:
    href = li.find('a')['href']
    sub_req = requests.get(base_url + href)
    sub_soup = BeautifulSoup(sub_req.text,'html.parser')

    title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
    info_lis = sub_soup.select('.info_list > li')
    info_dict = {}
    for li in info_lis:
        key = li.find('strong').get_text().split(' ')[0]
        val = li.find('span').get_text().strip()
        info_dict[key] =val

    author = info_dict['저자']
    try:
        translator = info_dict['번역']
    except:
        translator = '-'
    date = info_dict['출간']
    page = int(info_dict['페이지'].split(' ')[0].replace(',', ''))
    price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',', ''))
    lines.append([title, author,translator, date, page, price])

### 4. 데이터프레임으로 만들고 확인하기


In [43]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','쪽수','판매가'])
df

,제목,저자,번역,출간일,쪽수,판매가
0,포르잔 C++ 에센셜,"Behrouz A. Forouzan , Richard F. Gilberg",윤인성,2020-11-10,756,32000
1,플랫폼 비즈니스를 위한 경영정보시스템,"정창근 , 김종락 , 김시영",-,2020-10-20,522,29000
2,"IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습",김지연,-,2020-09-28,536,28000
3,뉴 머신시대의 기업가정신을 찾아서,이건창,-,2020-09-20,180,18000
4,세계화와 지역화 : 한국의 선택은?,황해두,-,2020-09-10,256,17000
5,포르잔 C++ 바이블,"Behrouz A. Forouzan , Richard F. Gilberg",윤인성,2020-08-31,1192,45000
6,4차산업혁명시대의 창업실무,"조보상 , 윤두열",-,2020-08-26,468,28000
7,딥러닝을 위한 선형대수학,Gilbert Strang,부산대학교 산업수학센터(FFMIMC),2020-08-05,560,38000
8,"IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습(2판)","박우창 , 남송휘 , 이현룡",-,2020-07-30,600,29000
9,대학을 버려야 대학이 산다,조동성 외 인천대학교 가족 178명,-,2020-07-25,216,18000


### 5. 10 페이지 정보 가져오기 

In [44]:
lines = []
for page in tqdm(range(1, 11)):
    url = base_url + sub_url + '?page=' + str(page)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    lis = soup.select('li.sub_book_list')

    for li in lis:
        href = li.find('a')['href']
        sub_req = requests.get(base_url + href)
        sub_soup = BeautifulSoup(sub_req.text,'html.parser')

        title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
        info_lis = sub_soup.select('.info_list > li')
        info_dict = {}
        for li in info_lis:
            key = li.find('strong').get_text().split(' ')[0]
            val = li.find('span').get_text().strip()
            info_dict[key] =val
          
        author = info_dict['저자']
        try:
            translator = info_dict['번역']
        except:
            translator = '-'
        date = info_dict['출간']
        page = int(info_dict['페이지'].split(' ')[0].replace(',', ''))
        price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',', ''))
        lines.append([title, author,translator, date, page, price]) 

  0%|          | 0/10 [00:00<?, ?it/s]

### 6. 모든 데이터를 데이터 프레임으로 만들고 저장하기 

In [45]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','쪽수','판매가'])
df.tail()

,제목,저자,번역,출간일,쪽수,판매가
195,"IT Cookbook, 처음 만나는 전자회로",황형수,-,2019-07-10,440,24000
196,스트랭 미분방정식과 선형대수학,Gilbert Strang,한빛수학교재연구소,2019-07-09,552,30000
197,"IT CookBook, 네트워크 개론(3판)",진혜진,-,2019-07-05,516,25000
198,"IT CookBook, 텐서플로로 배우는 딥러닝","박혜정 , 석경하 , 심주용 , 황창하",-,2019-06-30,448,27000
199,"IT CookBook, 파워포인트 2016",김지연,-,2019-06-29,420,19000


In [46]:
df.to_csv('한빛신간도서(220808).csv', index=False)